In [1]:
from pymongo import MongoClient
from pytz import timezone
from datetime import datetime
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import SGDClassifier
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
import matplotlib
from scipy.stats import scoreatpercentile
from scipy.spatial.distance import euclidean

%matplotlib inline



In [2]:
def emptyHourTable():
    return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


def getEmotion(tweet):
    emotion = tweet["groups"][0]["name"]
    return emotion

def getLocalTime(time, zone):
    zoneTable = {"International Date Line West":"Pacific/Midway","Midway Island":"Pacific/Midway","American Samoa":"Pacific/Pago_Pago","Hawaii":"Pacific/Honolulu","Alaska":"America/Juneau","Pacific Time (US & Canada)":"America/Los_Angeles","Tijuana":"America/Tijuana","Mountain Time (US & Canada)":"America/Denver","Arizona":"America/Phoenix","Chihuahua":"America/Chihuahua","Mazatlan":"America/Mazatlan","Central Time (US & Canada)":"America/Chicago","Saskatchewan":"America/Regina","Guadalajara":"America/Mexico_City","Mexico City":"America/Mexico_City","Monterrey":"America/Monterrey","Central America":"America/Guatemala","Eastern Time (US & Canada)":"America/New_York","Indiana (East)":"America/Indiana/Indianapolis","Bogota":"America/Bogota","Lima":"America/Lima","Quito":"America/Lima","Atlantic Time (Canada)":"America/Halifax","Caracas":"America/Caracas","La Paz":"America/La_Paz","Santiago":"America/Santiago","Newfoundland":"America/St_Johns","Brasilia":"America/Sao_Paulo","Buenos Aires":"America/Argentina/Buenos_Aires","Montevideo":"America/Montevideo","Georgetown":"America/Guyana","Greenland":"America/Godthab","Mid-Atlantic":"Atlantic/South_Georgia","Azores":"Atlantic/Azores","Cape Verde Is.":"Atlantic/Cape_Verde","Dublin":"Europe/Dublin","Edinburgh":"Europe/London","Lisbon":"Europe/Lisbon","London":"Europe/London","Casablanca":"Africa/Casablanca","Monrovia":"Africa/Monrovia","UTC":"Etc/UTC","Belgrade":"Europe/Belgrade","Bratislava":"Europe/Bratislava","Budapest":"Europe/Budapest","Ljubljana":"Europe/Ljubljana","Prague":"Europe/Prague","Sarajevo":"Europe/Sarajevo","Skopje":"Europe/Skopje","Warsaw":"Europe/Warsaw","Zagreb":"Europe/Zagreb","Brussels":"Europe/Brussels","Copenhagen":"Europe/Copenhagen","Madrid":"Europe/Madrid","Paris":"Europe/Paris","Amsterdam":"Europe/Amsterdam","Berlin":"Europe/Berlin","Bern":"Europe/Berlin","Rome":"Europe/Rome","Stockholm":"Europe/Stockholm","Vienna":"Europe/Vienna","West Central Africa":"Africa/Algiers","Bucharest":"Europe/Bucharest","Cairo":"Africa/Cairo","Helsinki":"Europe/Helsinki","Kyiv":"Europe/Kiev","Riga":"Europe/Riga","Sofia":"Europe/Sofia","Tallinn":"Europe/Tallinn","Vilnius":"Europe/Vilnius","Athens":"Europe/Athens","Istanbul":"Europe/Istanbul","Minsk":"Europe/Minsk","Jerusalem":"Asia/Jerusalem","Harare":"Africa/Harare","Pretoria":"Africa/Johannesburg","Kaliningrad":"Europe/Kaliningrad","Moscow":"Europe/Moscow","St. Petersburg":"Europe/Moscow","Volgograd":"Europe/Volgograd","Samara":"Europe/Samara","Kuwait":"Asia/Kuwait","Riyadh":"Asia/Riyadh","Nairobi":"Africa/Nairobi","Baghdad":"Asia/Baghdad","Tehran":"Asia/Tehran","Abu Dhabi":"Asia/Muscat","Muscat":"Asia/Muscat","Baku":"Asia/Baku","Tbilisi":"Asia/Tbilisi","Yerevan":"Asia/Yerevan","Kabul":"Asia/Kabul","Ekaterinburg":"Asia/Yekaterinburg","Islamabad":"Asia/Karachi","Karachi":"Asia/Karachi","Tashkent":"Asia/Tashkent","Chennai":"Asia/Kolkata","Kolkata":"Asia/Kolkata","Mumbai":"Asia/Kolkata","New Delhi":"Asia/Kolkata","Kathmandu":"Asia/Kathmandu","Astana":"Asia/Dhaka","Dhaka":"Asia/Dhaka","Sri Jayawardenepura":"Asia/Colombo","Almaty":"Asia/Almaty","Novosibirsk":"Asia/Novosibirsk","Rangoon":"Asia/Rangoon","Bangkok":"Asia/Bangkok","Hanoi":"Asia/Bangkok","Jakarta":"Asia/Jakarta","Krasnoyarsk":"Asia/Krasnoyarsk","Beijing":"Asia/Shanghai","Chongqing":"Asia/Chongqing","Hong Kong":"Asia/Hong_Kong","Urumqi":"Asia/Urumqi","Kuala Lumpur":"Asia/Kuala_Lumpur","Singapore":"Asia/Singapore","Taipei":"Asia/Taipei","Perth":"Australia/Perth","Irkutsk":"Asia/Irkutsk","Ulaanbaatar":"Asia/Ulaanbaatar","Seoul":"Asia/Seoul","Osaka":"Asia/Tokyo","Sapporo":"Asia/Tokyo","Tokyo":"Asia/Tokyo","Yakutsk":"Asia/Yakutsk","Darwin":"Australia/Darwin","Adelaide":"Australia/Adelaide","Canberra":"Australia/Melbourne","Melbourne":"Australia/Melbourne","Sydney":"Australia/Sydney","Brisbane":"Australia/Brisbane","Hobart":"Australia/Hobart","Vladivostok":"Asia/Vladivostok","Guam":"Pacific/Guam","Port Moresby":"Pacific/Port_Moresby","Magadan":"Asia/Magadan","Srednekolymsk":"Asia/Srednekolymsk","Solomon Is.":"Pacific/Guadalcanal","New Caledonia":"Pacific/Noumea","Fiji":"Pacific/Fiji","Kamchatka":"Asia/Kamchatka","Marshall Is.":"Pacific/Majuro","Auckland":"Pacific/Auckland","Wellington":"Pacific/Auckland","Nuku'alofa":"Pacific/Tongatapu","Tokelau Is.":"Pacific/Fakaofo","Chatham Is.":"Pacific/Chatham","Samoa":"Pacific/Apia"}
    offset = timezone(zoneTable[zone]).utcoffset(datetime.now())
    return time + offset

def getTweetHour(tweet):
    zone = tweet["user"]["time_zone"]
    return getLocalTime(tweet["created_at"], zone).hour



def getTweetSentimentHour(tweet):
    zone = tweet["user"]["time_zone"]
    return getLocalTime(tweet["created_at"], zone).hour





def getUserEmotionFrequency(collectionName, timeZone=True, regularization=True, percentage = False):

    client = MongoClient('localhost', 27017)
    collection = client['idea'][collectionName]
    userFrequency = {}

    for tweet in collection.find():
        zone = tweet["user"]["time_zone"]
       
        if zone is not None:
            userID = tweet["user"]["id"]
            hour = getTweetHour(tweet)
            
            if userID not in userFrequency:
                userFrequency[userID] = {}
            if emotion not in userFrequency[userID]:
                    userFrequency[userID][emotion] = emptyHourTable()
                    
            userFrequency[userID][emotion][hour] += 1
    
   
    if regularization:
        for userID in userFrequency:
            total = 0
            for emotion in userFrequency[userID]:
                total += sum(userFrequency[userID][emotion])
            
            if percentage:
                total /= 100
            for emotion in userFrequency[userID]:
                for i in range(24):
                    userFrequency[userID][emotion][i] /=  total
            
    return userFrequency

 


def getUserFrequency(collectionName, timeZone=True, regularization=True, percentage = False):

    client = MongoClient('localhost', 27017)
    collection = client['idea'][collectionName]
    userFrequency = {}

    for tweet in collection.find():
        zone = tweet["user"]["time_zone"]
        if zone is not None:
            userID = tweet["user"]["id"]
            hour = getTweetHour(tweet)
            
            if userID not in userFrequency:
                userFrequency[userID] = emptyHourTable()
            userFrequency[userID][hour] += 1
    
    if regularization:
        for userID in userFrequency:
            total = sum(userFrequency[userID])
            if percentage:
                total /= 100
            for i in range(24):
                userFrequency[userID][i] /=  total
            
    return userFrequency


def frequencyPlot(userFrequency):
   
    X = []
    Y = []
    for hourTable in userFrequency.values():
        for i in range(24):
            if hourTable[i] < 0.3:
                X.append(i)
                Y.append(hourTable[i])
    X = np.array(X)
    Y = np.array(Y)
    fig, ax = plt.subplots()
    fit = np.polyfit(X, Y, deg=1)
    ax.plot(X, fit[0] * X + fit[1], color='red')
    ax.scatter(X, Y)

    fig.show()

    
    

 

def averagePlot(userFrequency, title):
    
    X = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
    Y = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for hourTable in userFrequency.values():
        for i in range(24): 
            Y[i] += hourTable[i]
    
    X = np.array(X)
    Y = np.array(Y)
    Y /= len(userFrequency)
    fig, ax = plt.subplots()
    fit = np.polyfit(X, Y, deg=3)
    ax.plot(X, fit[0] * X**3+ + fit[1]*X**2+fit[2]*X+fit[3], color='red')
    ax.scatter(X, Y)
    ax.set_xlabel("Hour",  fontsize=14)
    ax.set_ylabel("Probablity to Post",  fontsize=14)
    fig.suptitle("{} {} people".format(len(userFrequency), title),  fontsize=18)
    
    fig.show()   

  

def personalPlot(userHour):  
        
    X = []
    Y = []

    for i in range(24):
        if userHour[i] > 0:
            X.append(i)
            Y.append(userHour[i])
            
    X = np.array(X)
    Y = np.array(Y)
    fig, ax = plt.subplots()
   
    fit = np.polyfit(X, Y, deg=2)
    ax.plot(X, fit[0] * X**2+ + fit[1]*X+fit[2], color='red')
    ax.scatter(X, Y)
    plt.axis((0,24,0,0.2))
    fig.show()
  
    
    
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    return sqrt(LB_sum)

In [3]:

def getUserEmotionsCompositions(collectionName, percentage=True):

    client = MongoClient('localhost', 27017)
    collection = client['idea'][collectionName]
    emotionsComposition = {"joy":.0,"sadness": .0,"fear":.0, "anticipation": .0, "anger":.0, "trust": .0, "disgust": .0 ,"surprise" : .0}
    emotions = emotionsComposition.keys()
    usersEmotionsCompositions = {}
    for tweet in collection.find():
    
        userID = tweet["user"]["id"]
        emotion =  tweet["emotion"]["groups"][0]["name"]
            
        if userID not in usersEmotionsCompositions:
              usersEmotionsCompositions[userID] = {"joy":.0,"sadness": .0,"fear":.0, "anticipation": .0, "anger":.0, "trust": .0, "disgust": .0 ,"surprise" : .0}
        usersEmotionsCompositions[userID][emotion] += 1
    

        for userID in usersEmotionsCompositions:
            total = sum(usersEmotionsCompositions[userID].values())
            if percentage:
                total /= 100
            for emotion in emotions:
                usersEmotionsCompositions[userID][emotion] /=  total
            
    return usersEmotionsCompositions

In [4]:
def compositions2vector(usersEmotionsCompositions):
    vectors = np.zeros((len(usersEmotionsCompositions),8),dtype=float)
    emotions = ['surprise', 'fear', 'sadness', 'disgust', 'trust', 'anticipation', 'anger','joy']
    for i, composition in enumerate(usersEmotionsCompositions.values()):
        vector = [composition[emotion] for emotion in emotions]
        vectors[i,:] = vector
    return vectors

def frequency2vector(usersFrequency):
    emotions = ['surprise', 'fear', 'sadness', 'disgust', 'trust', 'anticipation', 'anger','joy']
    vectors = np.zeros((len(usersFrequency),len(emotions) * 24),dtype=float)
    for i, composition in enumerate(usersFrequency.values()):
        vector = np.ravel([composition.get(emotion, np.zeros(24, dtype=float)) for emotion in emotions])
        vectors[i,:] = vector
        
    return vectors
    
    
def compositions2polar(usersEmotionsCompositions):
    pos_emotions = ['joy', 'trust', 'anticipation']
    neg_emotions = ['fear', 'sadness', 'anger','disgust']
    vectors = np.zeros((len(usersEmotionsCompositions),2), dtype=float)
    for i, composition in enumerate(usersFrequency.values()):
        vectors[i,0] = sum([composition[emotion] for emotion in pos_emotions])
        vectors[i,1] = sum([composition[emotion] for emotion in neg_emotions])
        
    return vectors


    
def frequency2polar(usersFrequency):
    pos_emotions = ['joy', 'trust', 'anticipation']
    neg_emotions = ['fear', 'sadness', 'anger','disgust']
    vectors = np.zeros((len(usersFrequency),48), dtype=float)
    for i, composition in enumerate(usersFrequency.values()):
        vectors[i][0:24] = np.sum([composition.get(emotion, np.zeros(24, dtype=float)) for emotion in pos_emotions], axis=0)
        vectors[i][24:] = np.sum([composition.get(emotion, np.zeros(24, dtype=float)) for emotion in neg_emotions], axis=0)
        
    return vectors

def vector2polar(vectors):
    emotions = ['surprise', 'fear', 'sadness', 'disgust', 'trust', 'anticipation', 'anger','joy']
    pos_emotions = ['joy', 'trust', 'anticipation']
    neg_emotions = ['fear', 'sadness', 'anger','disgust']
    polars = np.zeros((vectors.shape[0],48), dtype=float)
    pos_index = [emotions.index(emotion) for emotion in pos_emotions]
    neg_index = [emotions.index(emotion) for emotion in neg_emotions]
    
    polars[:,0:24] = np.sum([vectors[:,i*24:(i+1)*24] for i in pos_index], axis=0)
    polars[:,24:] = np.sum([vectors[:,i*24:(i+1)*24] for i in neg_index], axis=0)

        
    return polars


In [5]:
def timeSeriesDiff(timeSereies, method = "simple"):
    left_shift = np.roll(timeSereies, -1)
    if method is None:
        return timeSereies
    elif method == "simple":
        return left_shift - timeSereies
    elif method == "division":
        return left_shift / timeSereies
    elif method == "log_return":
        return np.log((left_shift - timeSereies) / left_shift)
    elif method == "raw_return":
        return (left_shift - timeSereies) / left_shift
    


   
            
    
    
def averageEmotionPlot(vectors, title, method = None, style = "emotions",removal = "ocsvm"):
    if style == "emotions":
    
        colorTable = {"joy":"#FADB4D","sadness": "#729DC9","fear":"#35A450", "anticipation": "#F2993A", "anger":"#E43054", "trust": "#99CC33", "disgust": "#9F78BA" ,"surprise" : "#3FA5C0"}
        emotions = ['surprise', 'fear', 'sadness', 'disgust', 'trust', 'anticipation', 'anger','joy']
        colors = [colorTable[emotion] for emotion in emotions]
        
        
    elif style == "polar":
        emotions = ['POS',"NEG"]
        colorTable = {"POS":"green","NEG": "red"}

        colors = ['red','green']
        
    X = np.arange(24)
    
    fig, ax = plt.subplots()
    
    fig.set_size_inches(14,8,forward=True)
    fig.suptitle("{} {} people".format(vectors.shape[0], title),  fontsize=18, color="white")
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white') 
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.title.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.xaxis.label.set_color('white')
    ax.set_xlabel("Hour",  fontsize=14)
    ax.set_ylabel("Probablity to Post",  fontsize=14)
       

    
    for i,emotion in enumerate(emotions):

        slice = range(i*24, (i+1)*24)
        Y = np.mean(vectors[:, slice], axis=0)
        Y = timeSeriesDiff(Y, method)
    
        ax.plot(X, Y, color = colorTable[emotion], label=emotion,linewidth=10)
        ax.legend(loc=2)

    fig.show()   

In [6]:
def emotionPieChart(vectors, title):

  
    colorTable = {"joy":"#FADB4D","sadness": "#729DC9","fear":"#35A450", "anticipation": "#F2993A", "anger":"#E43054", "trust": "#99CC33", "disgust": "#9F78BA" ,"surprise" : "#3FA5C0"}
    emotions = ['surprise', 'fear', 'sadness', 'disgust', 'trust', 'anticipation', 'anger','joy']
    colors = [colorTable[emotion] for emotion in emotions]
   
    

    vectors_mean = np.mean(vectors,axis=0)

   
    fig, ax = plt.subplots()
    fig.set_size_inches(8,8,forward=True)
    
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white') 
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.title.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.xaxis.label.set_color('white')
    patches, texts, autotexts =  plt.pie(vectors_mean,labels=emotions, colors=colors, autopct='%1.1f%%')
    for text in texts:
        text.set_color('white')
    fig.suptitle("{} {} people".format(vectors.shape[0], title),  fontsize=18, color="white")
    plt.show()

In [7]:
def outliers_removal(vectors, fraction=10, method="ocsvm"):
    if method == "ocsvm":
        ocsvm = OneClassSVM()
        ocsvm.fit(vectors)
        decision_function = ocsvm.decision_function(vectors).ravel()
        threshold = scoreatpercentile(decision_function, fraction)
        prediction = vectors[decision_function > threshold]
        
    elif method == "euclidean":
        distances = np.zeros(vectors.shape[0],dtype=float)
        mean = np.mean(vectors)
        
        for i, vector in enumerate(vectors):
            distances[i] = euclidean(vector, mean)
            
        threshold = scoreatpercentile(distances, 100 - fraction)
        prediction = vectors[distances < threshold]
        
    return prediction

In [8]:
def detrendPlot(vectors, title, method = None, style = "polar",removal = None, polynomial=5):
    if style == "emotions":
    
        colorTable = {"joy":"#FADB4D","sadness": "#729DC9","fear":"#35A450", "anticipation": "#F2993A", "anger":"#E43054", "trust": "#99CC33", "disgust": "#9F78BA" ,"surprise" : "#3FA5C0"}
        emotions = ['surprise', 'fear', 'sadness', 'disgust', 'trust', 'anticipation', 'anger','joy']
        colors = [colorTable[emotion] for emotion in emotions]
    elif style == "polar":
        emotions = ['POS',"NEG"]
        colorTable = {"POS":"green","NEG": "red","POS residual":"#34aa34","NEG residual":"#f72a2a", }

        colors = ['red','green']
        
    if removal is not None:
        vectors = outliers_removal(vectors, method=removal)
        vectors = vector2polar(vectors)
    
    X = np.arange(24)
    fig, ax = plt.subplots()
    
    fig.set_size_inches(14,8,forward=True)
    fig.suptitle("{} {} people".format(vectors.shape[0], title),  fontsize=18, color="white")
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white') 
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.title.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.xaxis.label.set_color('white')
    ax.set_xlabel("Hour",  fontsize=14)
    ax.set_ylabel("Probablity to Post",  fontsize=14)
       

    
    for i,emotion in enumerate(emotions):

        slice = range(i*24, (i+1)*24)
        Y = np.mean(vectors[:, slice], axis=0)
        Y = timeSeriesDiff(Y, method)
        model = np.polyfit(X, Y, polynomial)
        predicted = np.polyval(model, X)
        ax.plot(X, Y, color = colorTable[emotion], label=emotion, linewidth=5)
        ax.plot(X, predicted, color = colorTable[emotion], label=(emotion + " trend"), linestyle="-.", linewidth=5)
        ax.plot(X, Y-predicted, color = colorTable[emotion+" residual"], label=(emotion + " residual"), linewidth=5)

        ax.legend(loc=2)


In [9]:
regularUserCompositions = getUserEmotionsCompositions("regularUser_en_fixed_emotion")
BPDUserCompositions = getUserEmotionsCompositions("BPD_fixed_emotion")
regularUserFrequency = getUserEmotionFrequency("regularUser_en_fixed_emotion")
BPDUserFrequency = getUserEmotionFrequency("BPD_fixed_emotion")

UnboundLocalError: local variable 'emotion' referenced before assignment

In [ ]:
regularPolar = frequency2polar(regularUserFrequency)
detrendPlot(regularPolar, title="Normal", polynomial=5)


In [ ]:
BPDPolar = frequency2polar(BPDUserFrequency)
detrendPlot(BPDPolar, title="Normal", polynomial=5)